# Python + SQL: Crear y modificar bases de datos

## Crear una base de datos usando MySQL Connector/Python:

Para crear una base de datos MySQL desde Python, podemos ejecutar la siguiente sentencia usando un cursor y el método *execute()*:

In [2]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE BD_pruebas")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)


1007 (HY000): Can't create database 'bd_pruebas'; database exists
Error Code: 1007
SQLSTATE HY000
Message Can't create database 'bd_pruebas'; database exists


Al igual que hacíamos en la lección anterior con las consultas SELECT, para crear una base de datos hemos creado un connector usando *connect()* pasándole el user, la password y el host del servidor MySQL al que queremos conectarnos. En este caso, al querer crear una base de datos nueva, no hemos indicado a qué base de datos nos queríamos conectar.

Posteriormente, se ha creado un cursor para ejecutar la sentencia SQL que creará la base de datos de nombre BD_pruebas. Llamando al método *execute()* se envía el comando al servidor.

En este caso, vamos a aprovechar para introducir el manejo de excepciones a la hora de trabajar con MySQL Connector/Python. Todas las clases de excepción que pueden darse al trabajar con esta librería se encuentran en el paquete mysql.connector.errors. Podemos usarlo para capturar y mostrar la información acerca de los errores que sucedan.

Si ejecutamos dos veces seguidas el código anterior, la segunda vez que el programa intente crear la base de datos, recibirá un mensaje del servidor diciendo que ya existe una base de datos con ese mismo nombre. Debido a esto, el programa terminará con un error. Usando *except* y *mysql.connector.Error* podemos mostrar por pantalla más información acerca del error. Una segunda ejecución del código devolverá el error: "Can't create database 'bd_pruebas'; database exists".

## Crear una tabla usando MySQL Connector/Python:

Para crear una tabla en MySQL desde Python, podemos ejecutar la siguiente sentencia usando un cursor y el método *execute()*:

In [4]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)


1050 (42S01): Table 'customers' already exists
Error Code: 1050
SQLSTATE 42S01
Message Table 'customers' already exists


De la misma manera que al intentar crear una base de datos, en este caso se vuelve necesario el tratamiento de excepciones ya que no se pueden crear dos tablas con el mismo nombre en una base de datos. Una segunda ejecución del código devolverá el error: "Table 'customers' already exists".

## Alterando una tabla ya existente usando MySQL Connector/Python:

Una vez ya hemos definido una tabla con CREATE, puede ser necesario cambiar su estructura para añadir o eliminar columnas o restricciones a las mismas. Realizaremos esta tarea con el uso de un cursor y *execute()*:

In [6]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
try:
    mycursor.execute("ALTER TABLE customers ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1060 (42S21): Duplicate column name 'id'
Error Code: 1060
SQLSTATE 42S21
Message Duplicate column name 'id'


El anterior código añade una columna llamada *id* con tipo de dato número entero (int) a la tabla *customers*. Adicionalmente estamos definiendo varias restricciones o constraints para la columna. Estas son: que sea clave primaria y que se incremente automaticamente. De esta manera, el valor de ID nunca podrá ser igual para dos entradas de la tabla (PRIMARY KEY), y si no indicamos un valor a la hora de introducir un nuevo registro, a este se le asignará un valor único automáticamente (AUTO_INCREMENT).

De nuevo se hace necesario el manejo de excepciones ya que no podemos añadir a la misma tabla varias columnas con el mismo nombre. Una segunda ejecución del código devolverá el error: "Duplicate column name 'id'".

## Insertar datos en una tabla usando MySQL Connector/Python:

Ahora que ya hemos creado una base de datos, una tabla dentro de la misma, y definido algunas columnas junto con sus tipos de datos y restricciones, podemos comenzar a insertar registros en la misma:

In [7]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = ("Ana", "Calle 21")
try: 
    mycursor.execute(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1 registro insertado.


En el código anterior realizamos la conexión con la base de datos BD_pruebas de la manera habitual. Después de crear el cursor *mycursor*, pasamos a definir la sentencia SQL que usaremos para insertar los datos en la tabla. En este caso diferenciamos entre dos argumentos que le pasamos a *execute()*: la sintaxis de la sentencia SQL (*sql*) y los valores concretos (*val*). De esta manera podemos reutilizar la misma estructura del INSERT para guardar muchos valores en la tabla solo teniendo que cambiar el contenido de *val*.

**Uso de commit:** A la hora de trabajar con sentencias INSERT es necesario el uso de *commit()* para que los cambios se efectuen en la base de datos. De no llamar a ese método, las inserciones no se llevarán a cabo. Una manera de trabajo interesante es usar *execute()* para ejecutar todas las inserciones una tras otra y al terminar, llamar a *commit()* una sola vez. El uso de este método también nos permite realizar los cambios en la base de datos solo cuando estemos seguros de que todos ellos son correctos. 

**Inserción de múltiples registros:** Si quisieramos insertar múltiples filas en una misma tabla, podríamos definir todos los datos de las mismas dentro de la variable *val* en forma de un array de tuplas, para luego pasarla como argumento a *executemany()* (en vez de a *execute()*):


In [10]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = [
  ('Ana', 'Lowstreet 4'),
  ('Rocio', 'Apple st 652'),
  ('Juana', 'Mountain 21'),
  ('Pedro', 'Valley 345')
]

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

4 registro/s insertado/s.


**Uso de rollback:** Si después de haber realizado una transacción con *execute()* nos diésemos cuenta de que los datos que hemos introducido son incorrectos en alguna manera, si aún no hemos ejecutado *commit()* podríamos dar marcha atrás y desestimar los cambios usando *rollback()*:

In [11]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "INSERT INTO customers (name, address) VALUES (%s, %s)"
val = ("Lucia", "Plaza 22")
try:
    mycursor.execute(sql, val)
    cnx.rollback()
    print(mycursor.rowcount, "registro no insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

mycursor.execute("SELECT * FROM customers")  
results = mycursor.fetchall()
print(results)

1 registro no insertado.
[('Ana', 'Calle 21', 1), ('Ana', 'Lowstreet 4', 2), ('Rocio', 'Apple st 652', 3), ('Juana', 'Mountain 21', 4), ('Pedro', 'Valley 345', 5), ('Ana', 'Lowstreet 4', 7), ('Rocio', 'Apple st 652', 8), ('Juana', 'Mountain 21', 9), ('Pedro', 'Valley 345', 10)]


En el código anterior definimos la inserción, llamamos a *execute()* pero luego decidimos que no queremos implementar los cambios en la tabla. Para volver al estado anterior y desestimar los cambios, podemos usar *rollback()*. La llamada a este método eliminará los cambios propuestos por todos los *execute()* realizados desde la última llamada a *commit()*.

## Actualizar una tabla usando MySQL Connector/Python:

La librería también permite actualizar los datos de registros ya existentes en la tabla mediante la ejecución de sentencias UPDATE:

In [12]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "UPDATE customers SET address = 'Canyon 123' WHERE address = 'Valley 345'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s modificado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

mycursor.execute("SELECT * FROM customers")  
results = mycursor.fetchall()
print(results)

2 registro/s modificado/s.
[('Ana', 'Calle 21', 1), ('Ana', 'Lowstreet 4', 2), ('Rocio', 'Apple st 652', 3), ('Juana', 'Mountain 21', 4), ('Pedro', 'Canyon 123', 5), ('Ana', 'Lowstreet 4', 7), ('Rocio', 'Apple st 652', 8), ('Juana', 'Mountain 21', 9), ('Pedro', 'Canyon 123', 10)]


Este tipo de sentencias también necesitan de la ejecución de *commit()* para que los cambios se lleven a cabo. Igualmente, podemos usar *rollback()* para desestimarlos.

## Eliminar registros de una tabla usando MySQL Connector/Python:

Si queremos eliminar un registro de una tabla existente, se puede usar *execute()* con la sentencia SQL "DELETE FROM":

In [13]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "DELETE FROM customers WHERE address = 'Calle 21'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s eliminado/s")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1 registro/s eliminado/s


## Eliminar una tabla usando MySQL Connector/Python:

Por último, podemos eliminar una tabla entera usando *execute()* y "DROP TABLE":

In [15]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='BD_pruebas')


mycursor = cnx.cursor()
sql = "DROP TABLE customers"
try:
    mycursor.execute(sql)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1051 (42S02): Unknown table 'bd_pruebas.customers'
Error Code: 1051
SQLSTATE 42S02
Message Unknown table 'bd_pruebas.customers'


Si intentamos eliminar una tabla que no exista en la base de datos, la excepción contendrá un mensaje de error del tipo: Unknown table 'bd_pruebas.customers'

#### ENUNCIADO EJERCICIOS

En este conjunto de ejercicios vamos a volver a usar la tabla Customers (Clientes/as) que vamos a importar en MySQL Workbench. Si tienes dudas de como importarla, revisita la página asociada de tutorial.

La tabla Customers tiene las siguientes columnas:

**Customers**(`customerNumber`, `customerName`, `contactLastName`, `contactFirstName`, `phone`, `addressLine1`, `addressLine2`, `city`, `state`, `postalCode`, `country`, `salesRepEmployeeNumber`, `creditLimit`) 

Cada columna es bastante autodescriptiva en su nombre, pero vamos a incluir una pequeña descripción: 

- *customerNumber*: el número identificativo de las clientas/es. Es un número entero y sirve de clave primaria.
- *customerName*: el nombre de las empresas en las que trabajan las/los clientas/es. Es una cadena de texto.
- *contactLastName*: El apellido de la persona de contacto en la empresa cliente. Es una cadena de texto.
- *contactFirstName*: El nombre de la persona de contacto en la empresa cliente. Es una cadena de texto.
- *phone*: El teléfono de la persona de contacto en la empresa cliente. Es una cadena de texto (ya que hay espacios).
- *adressLine1*: La dirección (calle, número, etc.) de la empresa cliente. Es una cadena de texto.
- *adressLine2*: La dirección de la empresa cliente (si se necesita mas espacio). Es una cadena de texto. Muchas veces está vacía.
- *city*: La ciudad de la empresa cliente.
- *state*: El estado en el que se encuentra la empresa cliente. Válido para los Estados Unidos. Es una cadena de texto.
- *postalCode*: El código postal. Es una cadena de texto (ya que puede haber espacios).
- *country*: El país de la empresa cliente. Es una cadena de texto.
- *salesRepEmployeeNumber*: El número identificador de la empleada o empleado que lleva a esa empresa cliente. Es un número entero.
- *creditLimit*: El límite de crédito que tiene la empresa cliente. 

La tabla Employees tiene las siguientes columnas: 

- *employeeNumber*: el número identificativo de las empleadas/os. Es un número entero y sirve de clave primaria.
- *lastName*: el apellido de las empleadas. Es una cadena de texto.
- *firstName*: el nombre de las empleadas. Es una cadena de texto.
- *extension*: su extensión telefónica. Es una cadena de texto.
- *email*: el correo electrónico de la empleada. Es una cadena de texto.
- *officeCode*: El código de la oficina de la empleada. Es una cadena de texto.
- *reportsTo*: el número identificativo de la empleada a la que reporta (su supervisora). Es un número entero y clave foránea (relacionada con employeeNumber).
- *jobTitle*: el nombre del puesto de trabajo que desempeña. Es una cadena de texto.


Considera que la base de datos que contiene estas tablas se llama "tienda".

#### EJERCICIO 1

Crea una tabla Proyectos con información acerca de los proyectos de una empresa. Que al menos tenga los campos ID_Proyecto, NombreProyecto, Presupuesto.
Piensa cómo definir los tipos de datos de las columnas, clave primaria, clave foránea y otras restricciones.

In [37]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='tienda')

mycursor = cnx.cursor()

try:
    mycursor.execute("""CREATE TABLE proyectos (ID_Proyecto INT PRIMARY KEY AUTO_INCREMENT, NombreProyecto VARCHAR(200), Presupuesto FLOAT)""")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)



MySQLCursor: CREATE TABLE proyectos (ID_Proyecto INT ..


#### EJERCICIO 2
Crea una tabla llamada "EmpleadasYProyectos" que contenga información sobre proyectos a los que esten asociadas las empleadas. Haz que dicha tabla tenga dos atributos: ID_Empleada e ID_Proyecto.

In [28]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='tienda')

mycursor = cnx.cursor()

try:
    mycursor.execute("""CREATE TABLE EmpleadasYProyectos (ID_Empleada INT PRIMARY KEY AUTO_INCREMENT, ID_proyecto INT)""")
    
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: CREATE TABLE EmpleadasYProyectos (ID_Emp..


#### EJERCICIO 3

Inserta entradas en la tabla Proyectos (las que quieras, pero al menos 3):

In [38]:
sql = "INSERT INTO proyectos (ID_Proyecto, NombreProyecto, Presupuesto) VALUES (%s, %s, %s)"
val = [
  ('1', 'Adalab', '3456.4'),
  ('2', 'Movidas', '4576.23'),
  ('3', 'Cosas secretas', '5654.0'),
  ('4', 'Arbol de navidad', '76.09')
]

try:
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: INSERT INTO proyectos (ID_Proyecto, Nomb..


#### EJERCICIO 4

Elimina el último registro de los que insertaste en el ejercicio anterior:

In [39]:
mycursor = cnx.cursor()
sql = "DELETE FROM proyectos WHERE ID_Proyecto = '4'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s eliminado/s")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1 registro/s eliminado/s


#### EJERCICIO 5

Elimina la tabla EmpleadasYProyectos:

In [40]:

sql = "DROP TABLE EmpleadasYProyectos"
try:
    mycursor.execute(sql)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

#### EJERCICIO 6

Crea una base de datos llamada BD_Alumna:

In [41]:
try:
    mycursor.execute("""CREATE SCHEMA BD_Alumna""")
    
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

MySQLCursor: CREATE SCHEMA BD_Alumna


#### EJERCICIO 7

Actualiza la primera entrada de la tabla Proyectos cambiando el Presupuesto del proyecto con ID_Proyecto = 1 a 30000€:

In [42]:
sql = "UPDATE proyectos SET presupuesto = '30000' WHERE ID_Proyecto = '1'"
try:
    mycursor.execute(sql)
    cnx.commit()
    print(mycursor.rowcount, "registro/s modificado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1 registro/s modificado/s.


In [43]:
mycursor.close()
cnx.close()